## Imports

In [ ]:
# !pip install pdfplumber
# !pip install pandas

In [ ]:
import pdfplumber
import pandas as pd
from datetime import datetime
from functools import reduce
import os
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# ==========================================

## Functions

In [ ]:
def convert_to_df(csv, bank_name):
    df = pd.read_csv(csv, usecols=["Date", "Transaction Details", "Balance"])
    if bank_name == "hsbc":
        df = df[df["Transaction Details"].str.contains("B/F BALANCE")]
        df["Date"] = pd.to_datetime(df['Date']).dt.strftime("%Y-%m")
        return df
    if bank_name == 'sc':
        df = df[df["Transaction Details"].str.contains("BALANCE FROM PREVIOUS STATEMENT")]
        df["Date"] = pd.to_datetime(df['Date']).dt.strftime("%Y-%m")
        return df
    if bank_name == 'boc':
        df = df[df["Transaction Details"].str.contains("Balance Brought Forward")]
        df["Date"] = pd.to_datetime(df['Date']).dt.strftime("%Y-%m")
        return df
    return df

# ==========================================

## START HERE

In [ ]:
hsbc_df = convert_to_df(f'csv/hsbc.csv', 'hsbc')
sc_df = convert_to_df(f'csv/sc.csv', 'sc')
# boc_df = convert_to_df(f'csv/boc.csv', 'boc')

In [ ]:
hsbc_df[["Date", "Balance"]].reset_index(drop=True)

In [ ]:
sc_df[["Date", "Balance"]].reset_index(drop=True)

In [ ]:
dfs_to_merge = [hsbc_df[["Date", "Balance"]].reset_index(drop=True), sc_df[["Date", "Balance"]].reset_index(drop=True)]
merged_df = pd.concat(dfs_to_merge,axis=1,join='outer',ignore_index=True)
merged_df

In [ ]:
dfs_to_merge = [hsbc_df[["Date", "Balance"]], sc_df[["Date", "Balance"]]]

merged_df = reduce(lambda x, y: pd.merge(x, y, on = 'Date', suffixes=("_HSBC", "_SC")), dfs_to_merge)
merged_df["Combined Balance"] = merged_df["Balance_HSBC"] + merged_df["Balance_SC"]
merged_df.head(5)

In [ ]:
merged_df.to_csv("combined_statement.csv", index=False, mode='w+')